In [3]:
import numpy as np
import time
import copy
import sys
sys.path.append('/home/ziniu.wzn/BayesCard')
import pandas as pd
import time
import bz2
import pickle
import logging
import ast

from DataPrepare.join_data_preparation import JoinDataPreparator
from Models.pgmpy_BN import Pgmpy_BN
from Evaluation.utils import parse_query
from Evaluation.cardinality_estimation import parse_query_single_table

In [6]:
query = "SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND iClass = 0 AND iKorean >= 0 AND iKorean <= 1 AND iMay75880 = 0 AND iRagechld >= 0 AND iRagechld <= 4 AND iRrelchld = 0"
tab_query = parse_query_single_table(query, BN_J)


In [7]:
tab_query

{'iAvail': [0],
 'iClass': [0],
 'iKorean': [0, 1],
 'iMay75880': [0],
 'iRagechld': [4, 2, 0, 3, 1],
 'iRrelchld': [0]}

In [ ]:
import time
tic = time.time()
print(BN_J.query(tab_query))
print(time.time()-tic)

In [12]:
df = pd.read_csv("/home/ziniu.wzn/Census/Census.csv", header=0, sep=",")
df = df.drop("caseid", axis=1)
df = df.dropna(axis=0)
print(len(df))
print(df.columns)

2458285
Index(['dAge', 'dAncstry1', 'dAncstry2', 'iAvail', 'iCitizen', 'iClass',
       'dDepart', 'iDisabl1', 'iDisabl2', 'iEnglish', 'iFeb55', 'iFertil',
       'dHispanic', 'dHour89', 'dHours', 'iImmigr', 'dIncome1', 'dIncome2',
       'dIncome3', 'dIncome4', 'dIncome5', 'dIncome6', 'dIncome7', 'dIncome8',
       'dIndustry', 'iKorean', 'iLang1', 'iLooking', 'iMarital', 'iMay75880',
       'iMeans', 'iMilitary', 'iMobility', 'iMobillim', 'dOccup', 'iOthrserv',
       'iPerscare', 'dPOB', 'dPoverty', 'dPwgt1', 'iRagechld', 'dRearning',
       'iRelat1', 'iRelat2', 'iRemplpar', 'iRiders', 'iRlabor', 'iRownchld',
       'dRpincome', 'iRPOB', 'iRrelchld', 'iRspouse', 'iRvetserv', 'iSchool',
       'iSept80', 'iSex', 'iSubfam1', 'iSubfam2', 'iTmpabsnt', 'dTravtime',
       'iVietnam', 'dWeek89', 'iWork89', 'iWorklwk', 'iWWII', 'iYearsch',
       'iYearwrk', 'dYrsserv'],
      dtype='object')


In [ ]:
BN = Pgmpy_BN("census")
BN.build_from_data(df, algorithm="greedy", max_parents=2, ignore_cols=['id'], sample_size=10000)

In [5]:
#with open('check_points/Census_chow-liu.pkl', 'rb') as f:
 #   BN = pickle.load(f)
with open('check_points/Census_junction.pkl', 'rb') as f:
    BN_J = pickle.load(f)

In [10]:
def convert_to_pandas_query(query):
    query_str = ""
    for attr in query:
        query_str += attr
        val = query[attr]
        if type(val) != list:
            query_str += (" == " + str(val) + " and ")
        elif len(val) == 1:
            query_str += (" == " + str(val[0]) + " and ")
        else:
            query_str += (" in " + str(val) + " and ")
    query_str = query_str[:-5]
    return query_str

In [14]:
len(df.query(convert_to_pandas_query(tab_query)))

398593

In [22]:
from time import perf_counter
def evaluate_cardinality(BN, query_path, samp, ratio=0.01):
    # read all queries
    with open(query_path) as f:
        queries = f.readlines()
    latencies = []
    q_errors = []
    for query_no, query_str in enumerate(queries):
        cardinality_true = int(query_str.split("||")[-1])
        query_str = query_str.split("||")[0]
        print(f"Predicting cardinality for query {query_no}: {query_str}")
        
        query = parse_query_single_table(query_str.strip(), BN)
        query = convert_to_pandas_query(query)
        card_start_t = perf_counter()
        cardinality_predict = len(samp.query(query))*(1/ratio)
        if cardinality_predict is None:
            continue
        card_end_t = perf_counter()
        latency_ms = (card_end_t - card_start_t) * 1000
        if cardinality_predict == 0 and cardinality_true == 0:
            q_error = 1.0
        elif cardinality_predict == 0:
            cardinality_predict = 1
        elif cardinality_true == 0:
            cardinality_true = 1

        q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        print(f"latency: {latency_ms} and error: {q_error}")
        latencies.append(latency_ms)
        q_errors.append(q_error)
    return latencies, q_errors

In [25]:
query_path = "/home/ziniu.wzn/Census/cardinality/query_one_side.sql"
latencies, q_errors = evaluate_cardinality(BN_J, query_path, df.sample(n=int(len(df)*0.001)), 0.001)

Predicting cardinality for query 0: SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND iClass = 0 AND iKorean >= 0 AND iKorean <= 1 AND iMay75880 = 0 AND iRagechld >= 0 AND iRagechld <= 4 AND iRrelchld = 0
latency: 9.861607104539871 and error: 1.0286181643932533
Predicting cardinality for query 1: SELECT COUNT(*) FROM climate WHERE dIncome5 = 0 AND iOthrserv = 0 AND dPOB = 0 AND iRownchld = 0 AND iSex >= 0 AND iSex <= 1 AND dYrsserv = 0
latency: 6.4648352563381195 and error: 1.0070992804968129
Predicting cardinality for query 2: SELECT COUNT(*) FROM climate WHERE dAncstry1 >= 1 AND dAncstry1 <= 11 AND iEnglish = 0 AND iMarital = 0 AND iPerscare = 2 AND iRagechld >= 0 AND iRagechld <= 4 AND dRearning = 0 AND iRspouse >= 1 AND iRspouse <= 3 AND iYearsch >= 0 AND iYearsch <= 10
latency: 8.06272029876709 and error: 1.0481566115245144
Predicting cardinality for query 3: SELECT COUNT(*) FROM climate WHERE dIncome2 = 0 AND dIncome7 = 0 AND dRearning >= 0 AND dRearning <= 3 AND iSex = 1 AND iTm

latency: 6.996020674705505 and error: 1.0206259355132818
Predicting cardinality for query 49: SELECT COUNT(*) FROM climate WHERE dDepart = 0 AND iDisabl2 >= 1 AND iDisabl2 <= 2 AND iImmigr = 0 AND dIncome6 = 0 AND iKorean = 0 AND iMeans >= 0 AND iMeans <= 1 AND iMobility = 1 AND iOthrserv = 0 AND iRiders = 0 AND iRspouse >= 1 AND iRspouse <= 6
latency: 8.485265076160431 and error: 1.0595891544937024
Predicting cardinality for query 50: SELECT COUNT(*) FROM climate WHERE iDisabl2 >= 0 AND iDisabl2 <= 2 AND iImmigr = 0 AND dIncome3 = 0 AND dIncome6 >= 0 AND dIncome6 <= 1 AND iLang1 >= 1 AND iLang1 <= 2 AND iRownchld >= 0 AND iRownchld <= 1 AND iRrelchld = 0 AND iSubfam1 = 0 AND dWeek89 >= 0 AND dWeek89 <= 1 AND iWorklwk = 1
latency: 8.696462959051132 and error: 1.015253086419753
Predicting cardinality for query 51: SELECT COUNT(*) FROM climate WHERE iDisabl2 = 2 AND dHours = 0 AND iImmigr = 0 AND dIncome3 = 0 AND iLooking >= 0 AND iLooking <= 2 AND iMarital = 4 AND iMay75880 = 0 AND iMea

latency: 7.400978356599808 and error: 1.027021563342318
Predicting cardinality for query 105: SELECT COUNT(*) FROM climate WHERE iClass = 0 AND iDisabl2 >= 0 AND iDisabl2 <= 2 AND iSept80 = 0 AND iTmpabsnt = 0
latency: 5.372956395149231 and error: 1.0082247191011235
Predicting cardinality for query 106: SELECT COUNT(*) FROM climate WHERE dIncome5 = 0 AND dPOB = 0
latency: 4.30239737033844 and error: 1.0139026315789474
Predicting cardinality for query 107: SELECT COUNT(*) FROM climate WHERE dAge >= 5 AND dAge <= 6 AND dAncstry2 >= 1 AND dAncstry2 <= 2 AND iClass = 1 AND dPOB = 0 AND iSex = 1
latency: 5.798589438199997 and error: 1.0183458929326794
Predicting cardinality for query 108: SELECT COUNT(*) FROM climate WHERE dDepart >= 3 AND dDepart <= 5 AND dIncome6 = 0 AND iRelat2 = 0 AND iSex >= 0 AND iSex <= 1 AND iVietnam = 0 AND iWorklwk >= 0 AND iWorklwk <= 2
latency: 6.630048155784607 and error: 1.0066384106487558
Predicting cardinality for query 109: SELECT COUNT(*) FROM climate WHER

latency: 10.670289397239685 and error: 1.000747323324627
Predicting cardinality for query 134: SELECT COUNT(*) FROM climate WHERE dAncstry1 >= 3 AND dAncstry1 <= 11 AND iClass >= 0 AND iClass <= 1 AND iLang1 = 2 AND dPOB >= 0 AND dPOB <= 6 AND iRemplpar = 0 AND iSex >= 0 AND iSex <= 1 AND iVietnam = 0 AND iYearwrk >= 0 AND iYearwrk <= 6
latency: 8.178599178791046 and error: 1.0904134484325307
Predicting cardinality for query 135: SELECT COUNT(*) FROM climate WHERE iImmigr = 0 AND dIncome5 = 0 AND iSept80 = 0 AND iSex >= 0 AND iSex <= 1 AND iSubfam1 = 0 AND iVietnam = 0
latency: 6.161071360111237 and error: 1.007363221016562
Predicting cardinality for query 136: SELECT COUNT(*) FROM climate WHERE dIncome4 = 0 AND dIncome5 = 0 AND iMay75880 = 0 AND iRelat1 = 0 AND iWorklwk >= 1 AND iWorklwk <= 2
latency: 5.552526563405991 and error: 1.0393238089232373
Predicting cardinality for query 137: SELECT COUNT(*) FROM climate WHERE dAncstry1 >= 0 AND dAncstry1 <= 3 AND iClass >= 4 AND iClass <= 6

latency: 6.813477724790573 and error: 1.123406779661017
Predicting cardinality for query 162: SELECT COUNT(*) FROM climate WHERE dAncstry2 >= 1 AND dAncstry2 <= 2 AND iAvail = 0 AND iFeb55 = 0 AND dIncome2 >= 0 AND dIncome2 <= 1 AND dPoverty = 2 AND iRelat2 = 0 AND iRlabor >= 0 AND iRlabor <= 1 AND iRspouse = 0 AND iWorklwk >= 0 AND iWorklwk <= 1
latency: 8.387558162212372 and error: 1.0057977528089888
Predicting cardinality for query 163: SELECT COUNT(*) FROM climate WHERE dAncstry2 >= 1 AND dAncstry2 <= 2 AND iEnglish >= 0 AND iEnglish <= 2 AND iImmigr = 0 AND iMobillim >= 0 AND iMobillim <= 2 AND dWeek89 = 2
latency: 5.982443690299988 and error: 1.0117223113964686
Predicting cardinality for query 164: SELECT COUNT(*) FROM climate WHERE iFeb55 = 0 AND iFertil >= 4 AND iFertil <= 5 AND dIncome6 = 0 AND dRearning >= 0 AND dRearning <= 3
latency: 5.551639944314957 and error: 1.096464027545558
Predicting cardinality for query 165: SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND dIncome

...

latency: 9.688839316368103 and error: 1.114456953642384
Predicting cardinality for query 310: SELECT COUNT(*) FROM climate WHERE iClass = 1 AND iEnglish = 0 AND dIncome7 = 0 AND iLooking = 0 AND dRearning >= 0 AND dRearning <= 2 AND iSubfam1 = 0 AND dYrsserv = 0
latency: 6.4365454018116 and error: 1.1699917293688096
Predicting cardinality for query 311: SELECT COUNT(*) FROM climate WHERE dIncome2 = 0 AND dIncome3 = 0 AND iMobillim = 2 AND iYearsch >= 4 AND iYearsch <= 9
latency: 5.229216068983078 and error: 1.0200868691312655
Predicting cardinality for query 312: SELECT COUNT(*) FROM climate WHERE iLooking = 0 AND iOthrserv = 0 AND iYearwrk = 1
latency: 4.441894590854645 and error: 1.0023264942016057
Predicting cardinality for query 313: SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND dDepart = 4 AND iMobility = 1 AND dPOB = 0 AND iRemplpar >= 0 AND iRemplpar <= 112 AND dYrsserv = 0
latency: 6.105221807956696 and error: 1.087327306839502
Predicting cardinality for query 314: SELECT C

latency: 6.505019962787628 and error: 1.0384674871296318
Predicting cardinality for query 340: SELECT COUNT(*) FROM climate WHERE dHours >= 0 AND dHours <= 3 AND dIncome5 = 0 AND dIncome7 >= 0 AND dIncome7 <= 1 AND iMay75880 = 0 AND dRpincome >= 0 AND dRpincome <= 2
latency: 6.181560456752777 and error: 1.011190610567966
Predicting cardinality for query 341: SELECT COUNT(*) FROM climate WHERE iDisabl2 >= 0 AND iDisabl2 <= 2 AND iEnglish >= 0 AND iEnglish <= 3 AND dHispanic = 0 AND dIncome3 = 0 AND dIncome4 = 0 AND iLang1 = 2 AND iLooking >= 0 AND iLooking <= 2 AND dOccup >= 0 AND dOccup <= 2 AND iRemplpar = 0 AND iRspouse = 0 AND iYearsch >= 4 AND iYearsch <= 11 AND dYrsserv = 0
latency: 9.714499115943909 and error: 1.1140819964349375
Predicting cardinality for query 342: SELECT COUNT(*) FROM climate WHERE dAncstry1 = 1 AND iCitizen = 0 AND iClass >= 0 AND iClass <= 1 AND dHispanic = 0 AND dHour89 = 1 AND dIncome1 >= 0 AND dIncome1 <= 2 AND iMeans >= 0 AND iMeans <= 10 AND dRearning >=

latency: 8.84304940700531 and error: 1.1725
Predicting cardinality for query 367: SELECT COUNT(*) FROM climate WHERE iDisabl1 = 2 AND iEnglish = 0 AND dIncome6 = 0 AND dIncome7 = 0 AND dIndustry >= 4 AND dIndustry <= 7 AND iRagechld = 4 AND iRPOB >= 23 AND iRPOB <= 52 AND iRrelchld = 0 AND iSubfam2 = 0 AND iWWII = 0
latency: 8.289206773042679 and error: 1.060023850536637
Predicting cardinality for query 368: SELECT COUNT(*) FROM climate WHERE dAge >= 5 AND dAge <= 7 AND dIncome5 = 0 AND dPOB = 4 AND iSubfam1 = 0
latency: 5.077481269836426 and error: 1.1379338372754613
Predicting cardinality for query 369: SELECT COUNT(*) FROM climate WHERE dDepart = 0 AND iMarital >= 1 AND iMarital <= 4 AND dRearning >= 0 AND dRearning <= 2 AND iRemplpar >= 0 AND iRemplpar <= 121 AND iRrelchld = 0 AND iRspouse >= 1 AND iRspouse <= 6 AND iSex = 0 AND iVietnam = 0
latency: 7.725968956947327 and error: 1.0076045627376427
Predicting cardinality for query 370: SELECT COUNT(*) FROM climate WHERE dAge >= 2 AN

latency: 7.038246840238571 and error: 1.076405282832527
Predicting cardinality for query 423: SELECT COUNT(*) FROM climate WHERE iCitizen = 0 AND dIncome2 = 0 AND dIncome4 = 0 AND dWeek89 = 0 AND dYrsserv = 0
latency: 5.386147648096085 and error: 1.029092272202999
Predicting cardinality for query 424: SELECT COUNT(*) FROM climate WHERE iMobility >= 1 AND iMobility <= 2 AND dPoverty = 2 AND iSubfam2 = 0
latency: 5.094941705465317 and error: 1.007311793675697
Predicting cardinality for query 425: SELECT COUNT(*) FROM climate WHERE dAncstry2 = 1 AND dIncome1 = 0 AND dPwgt1 = 1 AND iWWII = 0
latency: 4.832867532968521 and error: 1.0555606407322655
Predicting cardinality for query 426: SELECT COUNT(*) FROM climate WHERE dIncome5 = 0 AND iMarital >= 0 AND iMarital <= 4 AND iMilitary = 4 AND dPwgt1 >= 0 AND dPwgt1 <= 1 AND iRagechld = 4
latency: 6.010785698890686 and error: 1.026143196817197
Predicting cardinality for query 427: SELECT COUNT(*) FROM climate WHERE dAge >= 5 AND dAge <= 6 AND i

latency: 7.610321044921875 and error: 1.0052628466678493
Predicting cardinality for query 452: SELECT COUNT(*) FROM climate WHERE iCitizen = 0 AND iFertil >= 1 AND iFertil <= 3 AND dIncome2 = 0 AND dIncome7 = 0 AND iSept80 = 0
latency: 5.559653043746948 and error: 1.01621778584392
Predicting cardinality for query 453: SELECT COUNT(*) FROM climate WHERE iFertil = 0 AND dHours >= 0 AND dHours <= 1 AND dIncome2 = 0 AND iLooking = 0 AND dPwgt1 >= 0 AND dPwgt1 <= 1 AND dRpincome >= 2 AND dRpincome <= 4 AND iRPOB = 10 AND iRrelchld = 0 AND iSex >= 0 AND iSex <= 1
latency: 8.022293448448181 and error: 1.1226284474048573
Predicting cardinality for query 454: SELECT COUNT(*) FROM climate WHERE dDepart >= 0 AND dDepart <= 3 AND dIncome4 >= 0 AND dIncome4 <= 1 AND dIncome7 >= 0 AND dIncome7 <= 1 AND iMobillim >= 0 AND iMobillim <= 2 AND iRelat1 = 0 AND iRownchld = 0 AND iWorklwk >= 0 AND iWorklwk <= 2 AND iYearsch >= 9 AND iYearsch <= 11
latency: 8.05925577878952 and error: 1.0341804949134357
Pre

In [26]:
print(np.mean(latencies))
for i in [50,90,95,99,100]:
    print(np.percentile(q_errors, i))

6.995719379912584
1.03015873802339
1.2121333333333335
1.7391499999999998
374.23999999999955
1703.0
